In [1]:
import string

In [2]:
ascii_characters = string.printable
ascii_characters 

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [3]:
ascii_characters = ascii_characters[:-15]
ascii_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@['

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time


driver = webdriver.Chrome()
driver.get('https://onlineasciitools.com/convert-ascii-to-image')

driver.maximize_window()
time.sleep(1)

driver.execute_script('window.scrollTo(0, 900);')

font_dropdown = driver.find_element(By.CSS_SELECTOR, "select.input-option[data-index='typeface']")
font_dropdown.click()

monospace_option = driver.find_element(By.CSS_SELECTOR, "option[value='monospace']")
monospace_option.click()

driver.execute_script('window.scrollTo(0, 100);')   

input_area = driver.find_element(By.CSS_SELECTOR, "textarea.data")
for character in ascii_characters:
    input_area.send_keys(character)
    
    time.sleep(1)
    
    save_btn = driver.find_elements(By.CSS_SELECTOR, "div[data-toggle='toggle-save-as']")[1]
    save_btn.click()
    
    time.sleep(2)
    
    download_btn = driver.find_elements(By.CSS_SELECTOR, "div.widget.widget-save-as")[1]
    download_btn.click()
    
    input_area.clear()

driver.close()

In [28]:
file_names = ['output-onlineasciitools.png'] + [f'output-onlineasciitools ({num}).png' for num in range(1, 85)]
file_dirs = [f'ascii_images/{file_name}' for file_name in file_names]

file_dirs[:5]

['ascii_images/output-onlineasciitools.png',
 'ascii_images/output-onlineasciitools (1).png',
 'ascii_images/output-onlineasciitools (2).png',
 'ascii_images/output-onlineasciitools (3).png',
 'ascii_images/output-onlineasciitools (4).png']

In [5]:
# because all the files are in the same order as 'ascii_characters' we can just iterante through them and associate them
# with their respective carachter

In [29]:
from PIL import Image
img = Image.open('ascii_images/output-onlineasciitools.png').convert('L')

img

In [30]:
list(img.getdata())[:10]

[255, 255, 255, 255, 255, 255, 255, 255, 255, 255]

In [31]:
len(list(img.getdata()))

1248

In [32]:
26*48

1248

In [11]:
# this means that each value corresponde to the brightness
# now to calculate the brightness of each character i am going to calculate the average of all the pixels

In [35]:
character_file_dict = {}

for x in range(85):
    character_file_dict[ascii_characters[x]] = file_dirs[x]
    
character_file_dict['0']

'ascii_images/output-onlineasciitools.png'

In [37]:
brightness_dict = {}

for character in character_file_dict:
    img = Image.open(character_file_dict[character]).convert('L')
    
    total = 0
    for x in list(img.getdata()):
        total += x
        
    brightness = total / len(list(img.getdata()))
    
    brightness_dict[character] = brightness
    
brightness_dict

{'0': 185.8966346153846,
 '1': 211.54967948717947,
 '2': 207.18429487179486,
 '3': 209.43830128205127,
 '4': 200.50721153846155,
 '5': 206.75,
 '6': 199.84375,
 '7': 216.78445512820514,
 '8': 187.89342948717947,
 '9': 199.87740384615384,
 'a': 204.8790064102564,
 'b': 197.48798076923077,
 'c': 220.5216346153846,
 'd': 198.15785256410257,
 'e': 205.54006410256412,
 'f': 210.05208333333334,
 'g': 180.97596153846155,
 'h': 203.3084935897436,
 'i': 215.7852564102564,
 'j': 207.03044871794873,
 'k': 206.22035256410257,
 'l': 215.7988782051282,
 'm': 197.97756410256412,
 'n': 210.91266025641025,
 'o': 206.58573717948718,
 'p': 197.16266025641025,
 'q': 197.75240384615384,
 'r': 222.02964743589743,
 's': 214.06089743589743,
 't': 212.2540064102564,
 'u': 211.21634615384616,
 'v': 217.78766025641025,
 'w': 202.88301282051282,
 'x': 212.86298076923077,
 'y': 204.24198717948718,
 'z': 214.39423076923077,
 'A': 197.57532051282053,
 'B': 186.7636217948718,
 'C': 210.2540064102564,
 'D': 193.117788

In [27]:
df_data = [{'c':x, 'b':brightness_dict[x]} for x in brightness_dict]

import pandas as pd

df = pd.DataFrame(df_data)
df.sort_values(by='b')

,c,b
83,@,166.431891
67,&,180.802083
16,g,180.975962
52,Q,182.008814
0,0,185.896635
...,...,...
73,",",238.416667
77,:,239.661058
68,',245.213141
74,-,245.334135


In [40]:
# now we need to normalize the data

b_max = 245.991186
b_min = 166.431891

norm_b_dict = {}

for c in brightness_dict:
    norm_b_dict[c] = round(((brightness_dict[c] - b_min) / (b_max - b_min)), 3)
    
norm_b_dict

{'0': 0.245,
 '1': 0.567,
 '2': 0.512,
 '3': 0.541,
 '4': 0.428,
 '5': 0.507,
 '6': 0.42,
 '7': 0.633,
 '8': 0.27,
 '9': 0.42,
 'a': 0.483,
 'b': 0.39,
 'c': 0.68,
 'd': 0.399,
 'e': 0.492,
 'f': 0.548,
 'g': 0.183,
 'h': 0.464,
 'i': 0.62,
 'j': 0.51,
 'k': 0.5,
 'l': 0.621,
 'm': 0.397,
 'n': 0.559,
 'o': 0.505,
 'p': 0.386,
 'q': 0.394,
 'r': 0.699,
 's': 0.599,
 't': 0.576,
 'u': 0.563,
 'v': 0.646,
 'w': 0.458,
 'x': 0.584,
 'y': 0.475,
 'z': 0.603,
 'A': 0.391,
 'B': 0.256,
 'C': 0.551,
 'D': 0.335,
 'E': 0.453,
 'F': 0.587,
 'G': 0.365,
 'H': 0.381,
 'I': 0.553,
 'J': 0.616,
 'K': 0.45,
 'L': 0.687,
 'M': 0.283,
 'N': 0.282,
 'O': 0.347,
 'P': 0.458,
 'Q': 0.196,
 'R': 0.369,
 'S': 0.483,
 'T': 0.644,
 'U': 0.429,
 'V': 0.489,
 'W': 0.283,
 'X': 0.433,
 'Y': 0.589,
 'Z': 0.457,
 '!': 0.793,
 '"': 0.878,
 '#': 0.357,
 '$': 0.263,
 '%': 0.314,
 '&': 0.181,
 "'": 0.99,
 '(': 0.663,
 ')': 0.664,
 '*': 0.793,
 '+': 0.73,
 ',': 0.905,
 '-': 0.992,
 '.': 1.0,
 '/': 0.77,
 ':': 0.92,
 '

In [52]:
def get_c(b_value):
    c = ('', 2)
    
    for x in norm_b_dict:
        r = abs(b_value - norm_b_dict[x])
        
        if r < c[1]:
            c = (x, r)
    
    return c[0]
    

In [54]:
get_c(1)

'.'